# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Phạm Tiến Đức
- MSSV: 21127598
- Lớp: 21CLC08

# Import

In [390]:
import pandas as pd
import numpy as np

# Import thêm dữ thư viện nếu cần
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score

# Đọc dữ liệu

In [391]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
x_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

x_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [392]:
# Cài đặt các hàm cần thiết ở đây
def calcuateMAE(y, y_real):
	return np.mean(np.abs(y_real - y))

def shuffle_data(x_data, y_data):
    shuffled_indices = np.arange(len(x_data))
    np.random.shuffle(shuffled_indices)

    shuffled_x = x_data.iloc[shuffled_indices]
    shuffled_y = y_data.iloc[shuffled_indices]

    return shuffled_x, shuffled_y

# result = []
# x_shuffled, y_shuffled = shuffle_data(x_train, y_train)

# for i in range(5):
# 	x_data = x_shuffled.iloc[:, 18 + i:19 + i]
# 	y_data = y_shuffled
# 	kf = KFold(n_splits=5)	# Create a cross-validation iterator that generates train-test splits for K-fold cross-validation
# 	model = LinearRegression()
# 	scores = cross_val_score(model, x_data, y_data, cv=kf, scoring='neg_mean_absolute_error')
# 	scores = -scores
# 	result.append(scores.mean())
# # Tìm ra đặc trưng tốt nhất
# best_feature = np.argmin(result) + 18
# print(np.argmin(result))


def cross_validation_for_features(x_data, y_data, features, n_fold=5):
    result = []
    x_shuffled, y_shuffled = shuffle_data(x_data, y_data)

    for i in range(len(features)):
        # Get the index of a feature in the dataframe
        feature_index = x_shuffled.columns.get_loc(features[i])
        x_data = x_shuffled.iloc[:, feature_index:feature_index + 1]
        y_data = y_shuffled
        # Create a cross-validation iterator that generates train-test splits for K-fold cross-validation
        kf = KFold(n_splits=n_fold)
        model = LinearRegression()
        scores = cross_val_score(model, x_data, y_data, cv=kf, scoring='neg_mean_absolute_error')
        
        scores = -scores
        result.append(scores.mean())
    return result

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [393]:
# Phần code cho yêu cầu 1a
x_data = x_train.iloc[:, 0:11]
model = LinearRegression()
model.fit(x_data, y_train)
y_pred = model.predict(x_test.iloc[:, 0:11])
print(model.coef_)

[-23183.32950765    702.76679172   1259.0187879  -99570.60814074
  18369.9624496    1297.53200035  -8836.727123      141.75993906
    145.74234652    114.64331342  34955.75040521]


In [394]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
print("MAE: ", calcuateMAE(y_pred, y_test))

MAE:  105052.52978823165


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)
$$\text{Salary} = -23183.329*{Gender} + 702.767*{10percentage} + 1259.019*{12percentage} - 99570.608*{CollegeTier} + 18369.962*{Degree} + 1297.532*{collegeGPA} - 8836.727*{CollegeCityTier} + 141.760*{English} + 145.742*{Logical} + 114.643*{Quant} + 34955.750*{Domain}$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [395]:
# Phần code cho yêu cầu 1b
personal_traits = ['conscientiousness',	'agreeableness', 'extraversion', 'nueroticism', 'openess_to_experience']
result = cross_validation_for_features(x_train, y_train, personal_traits, 5)

# Tìm ra đặc trưng tốt nhất
print("Best feature: ", personal_traits[np.argmin(result)])

# In ra các kết quả cross-validation như yêu cầu
print("\nCross-validation results: ")
for index, feature in enumerate(personal_traits):
    print(feature, ": ", "{:.3f}".format(result[index]))

Best feature:  nueroticism

Cross-validation results: 
conscientiousness :  124276.529
agreeableness :  123708.190
extraversion :  123881.932
nueroticism :  123618.223
openess_to_experience :  123806.913


In [396]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
feature_index = x_train.columns.get_loc(personal_traits[np.argmin(result)])
best_personality_feature_model = LinearRegression()
best_personality_feature_model.fit(x_train.iloc[:, feature_index:feature_index+1], y_train)
y_pred = best_personality_feature_model.predict(x_test.iloc[:, feature_index:feature_index+1])

In [397]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
print("MAE: ", calcuateMAE(y_pred, y_test))

MAE:  119361.91739987815


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [398]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

In [399]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện

In [400]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [401]:
# Trình bày các phần tìm ra mô hình

## Thử nghiệm, so sánh các mô hình

In [402]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

In [403]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện

In [404]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$